In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
from libs.common import get_images_in_dir, plot_images
from libs.calibration import CameraCalibrator


In [ ]:
# %matplotlib qt

### Calibrate camera

In [ ]:
calibration_data_dir = 'camera_cal/'
output_dir = "output_images/"

calibrator = CameraCalibrator(9, 6)

calibrate_images = get_images_in_dir(calibration_data_dir)
used_images = []

for image in calibrate_images:
    if calibrator.calibrate(image):
        used_images.append(image)


calibrator.finish_calibrate()

In [ ]:
target_images = calibrate_images

target_images = [
    calibrate_images[0],
    calibrate_images[7],
    calibrate_images[12],
    calibrate_images[15],
    calibrate_images[19],
]
fig = plt.figure(figsize=(10,5 * len(target_images)))
for image_id, image in enumerate(target_images):
    result = [image, calibrator.undistort(image)]
    plot_images(fig, result, image_id, len(target_images))
    
# plt.savefig(output_dir + 'undistort.png')

### Bird view

In [ ]:
def transform_image(image):
    return calibrator.process_image(image)[-1]

In [ ]:
from libs.calibration import apply_shape, get_transform_dst


In [ ]:
def get_transform_src(shape):
    x1, x2, x3, x4 = 0.1, 0.37, 0.65, 0.95
    y1, y2 = 0.7, 0.95
    return apply_shape(np.array([
        [x1, y2],
        [x2, y1],
        [x3, y1],
        [x4, y2],
    ]), shape)

shape=(720, 1280)
src = get_transform_src(shape)
dst = get_transform_dst(shape)
M = cv2.getPerspectiveTransform(src, dst)


def transform(image):
    shape = [image.shape[1], image.shape[0]]
    return cv2.warpPerspective(image, M, shape)

In [ ]:
%time
test_images_dir = 'test_images/'

test_images = get_images_in_dir(test_images_dir)
# test_images = test_images

fig = plt.figure(figsize=(10, 5* len(test_images)))
for image_id, image in enumerate(test_images):
    undistorted = calibrator.undistort(image)
    transformed = transform(undistorted)
    result = [undistorted, transformed]
    plot_images(fig, result, image_id, len(test_images))
# plt.savefig(output_dir + 'transform.png')